# Quinta lista - Problema grupal 3

## Componentes del grupo

* Pol Casacuberta Gil
* Joaquin Faraone Prieto
* Agnès Felip i Díaz
* Marta Granero i Martí
3. Sumemos de nuevo
Nota: Este problema solo puede ser resuelto por grupos que resolvieron el problema de grupo
3 de la lista 3.
Los resultados del experimento de suma usando k-vecinos más cercanos fueron un poco decep-
cionantes incluso transformando los atributos para una mejor representación. Las relaciones no
lineales entre algunos atributos fueron las culpables de estos resultados no óptimos. Ahora que
conocemos otros clasificadores, podemos probar si pueden hacer un mejor trabajo.
El escenario será el mismo, aprendiendo a sumar números de tres dígitos con ejemplos definidos
por seis atributos, los tres primeros atributos corresponden a los tres dígitos del primer número
y los tres últimos a los dígitos del segundo número. La salida tiene cuatro valores, uno para cada
dígito para cada número del resultado de la suma. Por ejemplo:

a) Dado que vamos a utilizar clasificadores más potentes, probablemente necesitemos menos
ejemplos para funcionar mejor, así que generad dos conjuntos de datos de 25.000 y 50.000
ejemplos. Dividid los conjuntos de datos en un conjunto de test y entrenamiento con una
proporción de 90 %/10 %.

b) Como recordaréis, este es un problema de clasificación de salida múltiple, por lo que ne-
cesita usar el objeto MultiOutputClassifier de scikit learn. Esto envolverá los clasifi-
cadores para que pueda realizar este tipo de clasificación.
Usad SVC para la clasificación con un kernel rbf con diferentes valores para C. Podéis usar
diferentes potencias de 10 para ese parámetro para ver dónde está el mejor acierto para los
datos de test en lugar de usar validación cruzada, os tardará mucho si la usáis. Aseguraos
de usar el parámetro n_jobs de los objetos de clasificación de salida múltiple para que las
cosas se hagan en paralelo.


c) Usad GradientBoostingClassifier con diferentes valores para el número de estima-
dores y la profundidad máxima de los árboles. La cantidad de estimadores no tiene que
ser muy grande, pero probablemente necesitaréis alrededor de 50 árboles con la tasa de
aprendizaje predeterminada. Además, los árboles demasiado poco profundos darán malos
resultados. Evaluad el acierto sobre el test como en el apartado anterior.
Podéis acceder a los clasificadores individuales del clasificador de salida múltiple con el
atributo estimator_. Obtened la importancia de los atributos para cada clasificador que
encontraréis en el atributo feature_importances e interpretad los resultados.

d) ¿Por qué creéis que los resultados de estos clasificadores son mejores que los del clasificador k vecinos más cercanos? ¿Por qué creéis que necesitamos menos ejemplos para estos resultados?